1. Join two datasets together

In [ ]:
# Assign the new dataset a name - "data"
# "inner_join" gives results (records) that are in both datasets that you are joining
# "dataset_55451009_person_df" This is the name of the first dataset
# "dataset_55451009_survey_df" This is the name of the second dataset
data <- inner_join(dataset_55451009_person_df,
                             dataset_55451009_survey_df) %>%
  mutate_if(is.list, as.character)

dim(data) # This shows the dimensions (rows, columns) of your dataset
head(data, 5) # This displays the top 5 records on your dataset
tail(data, 6) # This displays the bottom 6 records on your dataset

2. Calculate the age of the participants

    a) Age as of the date of the survey completion

In [ ]:
# "data2" - The name of the new dataset with the age variable
# "data" - The name of the dataset containing the information we need (years) for age calculation
# age is calculated by subtracting the "date_of_birth" variable found in the "data" dataset from the "survey_datetime" variable found in the "data" dataset
data2 <- data %>%
                mutate_if(is.list, as.character) %>%
                mutate(age = year(data$survey_datetime) - year(data$date_of_birth))
head(data2) # Output the first few records of the new dataset "data2"
head(data2$age, 5) # Output the first 5 records of the age variable

    b) Current Age - This will give the age as of today, without considering if the participants are alive or not

In [ ]:
data2 <- data %>%
                mutate_if(is.list, as.character) %>%
                mutate(age = year(today()) - year(data$date_of_birth))

3. Select the Columns you need for your analysis

    a) keep the wanted columns

In [ ]:
# "data3" - The name of the updated dataset with the only columns you want
# "data2" - The name of the dataset with the data we need
# Quote the columns you wish to keep
data3 <- data2[, c("person_id", "gender", "race", "ethnicity", "age", "question_concept_id", "question", "answer")]
head(data3)

    b) Drop the unwanted columns

In [ ]:
# "data3" - The name of the updated dataset with the only columns you want
# "data2" - The name of the dataset with the data we need
# The negative sign "-" indicates which columns to drop
data3 <- select(data2, -procedure_concept_id, -standard_concept_code, -visit_occurrence_concept_name,
                             -procedure_source_concept_id, -source_concept_name, -date_of_birth, -sex_at_birth)
head(data3)

4. Summary statistics 

    a) For numerical variables

In [ ]:
# "data3" - Name of the dataset
# "age" - Numerical variable of interest
summary(data3$age)

    b) For categorical variables

In [ ]:
# Use the "table" function
# "data3" - Name of your dataset
# replace "gender" with your variable of interest
gender_table <- table(data3$gender)
gender_table_sort <- sort(table(data3$gender), decreasing = TRUE) # Sort the results
gender_table_sort # Output the results

In [ ]:
# % representation of gender
prop.table(gender_table) * 100

    c) Custom table to analyze two variables

In [ ]:
# Use the "table" function
# "data3" - Name of your dataset
# replace "gender" and "race" with your variables of interest
table(data3$gender, data3$race)

5. Age Bins

In [ ]:
# "data3" name of your dataset
# "age_bins" new column being calculated, and appended to the "data3" dataset
# "breaks" - The age cuts you want
# "labels" - How you want to label your age bins
data3$age_bins <- cut(data3$age, 
                                breaks = c(0, 25, 50, 75, 100),
                                labels = c("0-25", "26-50", "51-75", "76+"),
                                include.lowest = TRUE)
head(data3)

6. Plots

In [ ]:
# First create a table with the variables you need to plot
# To plot the gender representation
gender_table <- table(data3$gender) # Create a table
gender_table_sorted <- sort(table(data3$gender), decreasing = TRUE) # Sort the table
gender_table_sorted # View the output

In [ ]:
# Second - do the actual plot
# Optional - Set scipen option to prevent scientific notation
options(scipen = 10)

# Optional - Adjust the plot margins to accommodate longer labels
# Inside the bracket: 10 reps on the bottom margin, 6 reps on the left, 4 reps on top, 2 reps on the right, and 0.1 reps inches added to each margin
par(mar = c(10, 6, 4, 2) + 0.1) 

# Create a bar plot using base R
# "gender_table_sorted" - Name of the table/data to be plotted
# "las = 2" - displays labels perpendicularly to the x-axis. 
# "las = 1" displays horizontally, "las = 3" displays both axis vertically, and "las = 0" (default) displays parallel to the x-axis
barplot(gender_table_sorted, main = "Bar Plot of Gender (Descending Order)", 
        xlab = "Categories", ylab = "Count", las = 2)
 

7. Subsetting a dataset 

In [ ]:
# You can subset the dataset to analyze a specific group
# Assuming you want to analyze participants in your dataset who were diagnosed
# Assign a name to the target group - In this case, we are targeting the group/records with ID = 836772
# Assign a name to the new dataset being created - "diagnosed data"
# "data3" - the dataset with the data needed
# "question_concept_id" - the name of the column of interest
# "diagnosed" name given to the target group IDs

diagnosed <- "836772"
diagnosed_data <- data3[data3$question_concept_id == diagnosed, ]
head(diagnosed_data)
dim(diagnosed_data)

8. Cleaning Records 

In [ ]:
# Some of the records in the dataset are too long and not easy to read/represent on a plot/table
# To solve this, we will create a new variable with shorter and more easily representable values
# Take an example of the "gender" variable
table(data3$gender)

In [ ]:
# Let us now clean these responses to shorter and easily representable responses
# Load necessary libraries if not already loaded
install.packages("dplyr") # used for data manipulation
library(dplyr)

# "data4" - Name of the new dataset
# "data3" - Name of the dataset with the data we need for the analysis
# "edited_gender" - New column with edited gender responses
data4 <- data3 %>%
  mutate(edited_gender = case_when
  (
    gender %in% c("Not man only, not woman only, prefer not to answer, or skipped",
                  "Gender Identity: Additional Options", "PMI: Skip",
                  "I prefer not to answer") ~ "Other/Skip",
    gender == "Gender Identity: Non Binary" ~ "Non Binary",
    gender == "Gender Identity: Transgender" ~ "Transgender",
    gender == "Male" ~ "Male",
    gender == "Female" ~ "Female",
    TRUE ~ as.character(gender) # Keep other responses as they are
  )) %>%
  select(person_id, edited_gender, race, ethnicity, age, question_concept_id, question, answer) 

head(data4) # Display the new dataset

In [ ]:
# Take a look at the new representation of the gender
table(data4$edited_gender)